In [1]:
import numpy as np
import matplotlib.pyplot as plt
import glob
#import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Input
import os
import seaborn as sns
from keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from keras.utils.np_utils import to_categorical
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.applications.vgg16 import preprocess_input, decode_predictions
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from tensorflow.keras.optimizers import SGD

# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
import tensorflow as tf
if tf.test.gpu_device_name():
   print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
   print("Please install GPU version of TF")

Default GPU Device: /device:GPU:0


In [4]:
from keras.models import load_model
from numpy.linalg import norm


# totalFiles = []
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Normal-rgb-named/Split_1_Norm_Train/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Normal-rgb-named/Split_1_Norm_Test/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Normal-rgb-named/Split_1_Norm_Valid/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Hemorrhage-rgb-named/Split_1_Hem_Train/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Hemorrhage-rgb-named/Split_1_Hem_Test/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Hemorrhage-rgb-named/Split_1_Hem_Valid/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Infarct-rgb-named/Split_1_Inf_Train/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Infarct-rgb-named/Split_1_Inf_Train/*.jpg")
# for f in files:
#     totalFiles.append(f)
# files = glob.glob("/home/user/Documents/Db-Stroke-Split/Infarct-rgb-named/Split_1_Inf_Train/*.jpg")
# for f in files:
#     totalFiles.append(f)
# print("TOTAL FILES : ",len(totalFiles))

model = VGG16(weights='imagenet')                                                             # download the resnet50 model with imagenet weights
model.layers.pop()                                                                               # pop the last layer (model structure)
model = Model(inputs=model.inputs, outputs=model.layers[-1].output)                              # pass inputs and outputs to new model(for weight files) 
# model = load_model("heatmap_model.h5")
# print("Model imported successfully")
model.trainable = False  

def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))                                       # load image convert to size 224 x 224
    img_array = image.img_to_array(img)                                                          # convert image to array for preprocessing
    expanded_img_array = np.expand_dims(img_array, axis=0)                                       # expand dimentions of array since model takes input in 3D for convolution operation 
    preprocessed_img = preprocess_input(expanded_img_array)                                      # preprocess input which is imported from keras based on resnet50 model
    features = model.predict(preprocessed_img)                                                   # apply predictions
    flattened_features = features.flatten()                                                      # flatten features to 1D array 
    # print(flattened_features.shape)
    normalized_features = flattened_features / norm(flattened_features)                          # normalize features 
    return normalized_features

ResourceExhaustedError: failed to allocate memory [Op:AddV2]

In [ ]:
print("lalitkaim")
import pickle
from tqdm import tqdm, tqdm_notebook

filenames = sorted(totalFiles)
feature_list = []
for i in tqdm_notebook(range(len(filenames))):
    feature_list.append(extract_features(filenames[i]))
from sklearn.decomposition import PCA
num_feature_dimensions=128
pca = PCA(n_components = num_feature_dimensions)
pca.fit(feature_list)
feature_list_compressed = pca.transform(feature_list)
pickle.dump(pca,open('/content/drive/My Drive/stroke/pca_128.pickle','wb'))

In [ ]:
from keras.utils.np_utils import to_categorical
le = LabelEncoder()

train_labels_num = le.fit_transform(train_labels)
test_labels_num = le.fit_transform(test_labels)
val_labels_num = le.fit_transform(val_labels)

y_train_cat = to_categorical(train_labels_num)
y_test_cat = to_categorical(test_labels_num)
y_val_cat = to_categorical(val_labels_num)

x_train_np = np.array(train_images)
x_test_np = np.array(test_images)
x_val_np = np.array(val_images)

x_train_pre = preprocess_input(x_train_np)
x_test_pre = preprocess_input(x_test_np)
x_val_pre = preprocess_input(x_val_np)

print(x_train_pre.shape)
print(len(y_train_cat))
print(len(y_test_cat))
print(len(y_val_cat))

In [ ]:
from sklearn.utils import shuffle

x_train, y_train = shuffle(x_train_pre, y_train_cat)
x_test, y_test = shuffle(x_test_pre, y_test_cat)
x_val, y_val = shuffle(x_val_pre, y_val_cat)

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator
# datagen = ImageDataGenerator(
#     featurewise_center=True,
#     featurewise_std_normalization=True,
#     rotation_range=20,
#     width_shift_range=0.2,
#     height_shift_range=0.2,
#     horizontal_flip=True,
#     validation_split=0.2)
# datagen.fit(x_train)
# len(x_train)
# x = datagen.flow(x_train, y_train)
# print(x.next())

In [ ]:
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))

# for layer in VGG_model.layers:
#   layer.trainable = False
VGG_model.summary()

In [ ]:
train_feature_extractor = VGG_model.predict(x_train)
train_features = train_feature_extractor.reshape(train_feature_extractor.shape[0], -1)

test_feature_extractor = VGG_model.predict(x_test)
test_features = test_feature_extractor.reshape(test_feature_extractor.shape[0], -1)

val_feature_extractor = VGG_model.predict(x_val)
val_features = val_feature_extractor.reshape(val_feature_extractor.shape[0], -1)

print(train_features.shape)
print(test_features.shape)
print(val_features.shape)

In [ ]:
model = Sequential()
inputs = Input(shape=np.shape(train_features[1]))
hidden = Dense(256, activation='relu')(inputs)
output = Dense(3, activation='softmax')(hidden)
model = Model(inputs=inputs, outputs=output)
print(model.summary())

In [ ]:
opt = SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(train_features, y_train_cat, batch_size=16 ,epochs=20, verbose=1, validation_data=(val_features, y_val_cat))

In [ ]:
predict_test_cat = model.predict(test_features)
predict_test_num = np.argmax(predict_test_cat, axis=1)
predict_test_lab = le.inverse_transform(predict_test_num)

y_test_num = np.argmax(y_test, axis=1)
y_test_lab = le.inverse_transform(y_test_num)

In [ ]:
print("Accuracy = ", metrics.accuracy_score(y_test_lab, predict_test_lab))

In [ ]:
cm = confusion_matrix(y_test_lab, predict_test_lab)
sns.heatmap(cm, annot=True)

# PCA Start Here

In [ ]:
from sklearn.decomposition import PCA

pca_test = PCA(n_components=500)
pca_test.fit(train_features)
plt.plot(np.cumsum(pca_test.explained_variance_ratio_))
plt.xlabel("Number of components")
plt.ylabel("Cum variance")

In [ ]:
n_PCA_components = 200
pca = PCA(n_components=n_PCA_components)
train_PCA = pca.fit_transform(train_features)
test_PCA = pca.transform(test_features)

In [ ]:
model = Sequential()
inputs = Input(shape = (n_PCA_components,))
hidden = Dense(256, activation='relu')(inputs)
outputs = Dense(3, activation='softmax')(hidden)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

In [ ]:
opt = SGD(learning_rate=0.0001, decay=1e-6, momentum=0.9)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

In [ ]:
model.fit(train_PCA, y_train_cat, batch_size=16 ,epochs=20, verbose=1)

In [ ]:
predict_test_cat = model.predict(test_PCA)
predict_test_num = np.argmax(predict_test_cat, axis=1)
predict_test_lab = le.inverse_transform(predict_test_num)

In [ ]:
print("Accuracy with PCA : ", metrics.accuracy_score(y_test_lab, predict_test_lab))

In [ ]:
cm = confusion_matrix(y_test_lab, predict_test_lab)
sns.heatmap(cm, annot=True)

In [ ]:
print(y_test_lab[:20])
print(predict_test_lab[:20])

# PCA plotting


In [ ]:
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler

# standardized_data = test_features
standardized_data = StandardScaler().fit_transform(test_features)
print(standardized_data.shape)

In [ ]:
pca = decomposition.PCA()
pca.n_components = 2
pca_data = pca.fit_transform(standardized_data)
print(pca_data.shape)

In [ ]:
import pandas as pd
import seaborn as sn
pca_data1 = np.vstack((pca_data.T, y_test_lab)).T
print(pca_data1.shape)
print(pca_data1[:30])
pca_df = pd.DataFrame(data=pca_data1, columns=("Component-1", "Component-2", "label"))
sn.FacetGrid(pca_df, hue="label", height=6).set_xticklabels([]).set_yticklabels([]).map(plt.scatter, "Component-1","Component-2").add_legend()
plt.show()

# t-SNE Plotting

In [ ]:
def fashion_scatter(x, colors):
    # choose a color palette with seaborn.
    num_classes = len(np.unique(colors))
    palette = np.array(sns.color_palette("hls", num_classes))

    # create a scatter plot.
    f = plt.figure(figsize=(8, 8))
    ax = plt.subplot(aspect='equal')
    sc = ax.scatter(x[:,0], x[:,1], lw=0, s=40, c=palette[colors.astype(np.int)])
    plt.xlim(-25, 25)
    plt.ylim(-25, 25)
    ax.axis('off')
    ax.axis('tight')

    # add the labels for each digit corresponding to the label
    txts = []

    for i in range(num_classes):

        # Position of each label at median of data points.

        xtext, ytext = np.median(x[colors == i, :], axis=0)
        txt = ax.text(xtext, ytext, str(i), fontsize=24)
        txt.set_path_effects([
            PathEffects.Stroke(linewidth=5, foreground="w"),
            PathEffects.Normal()])
        txts.append(txt)

    return f, ax, sc, txts

In [ ]:
from sklearn.manifold import TSNE
fashion_tsne = TSNE(random_state=100).fit_transform(train_features)

In [ ]:
import matplotlib.patheffects as PathEffects

y_train_num = np.argmax(y_train, axis=1)
y_train_lab = le.inverse_transform(y_train_num)
tl = le.fit_transform(y_train_lab)
fashion_scatter(fashion_tsne, tl)